In [1]:
import pandas as pd
datafile='/Users/z003zhj/desktop/Recommendation system/BX-CSV-Dump/Bx-Book-Ratings.csv'
data = pd.read_csv(datafile,sep=";", header=0, names=["user", "isbn", "ratings"], encoding='cp1252')

In [2]:
data.head()

,user,isbn,ratings
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [3]:
bookfile='/Users/z003zhj/desktop/Recommendation system/BX-CSV-Dump/Bx-Books.csv'
books=pd.read_csv(bookfile, sep=";", header=0, names=["isbn", "title", "author"], error_bad_lines=False, usecols=[0,1,2],index_col=0, encoding='cp1252')

In [4]:
books.head()

,title,author
isbn,,
0195153448,Classical Mythology,Mark P. O. Morford
0002005018,Clara Callan,Richard Bruce Wright
0060973129,Decision in Normandy,Carlo D'Este
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
0393045218,The Mummies of Urumchi,E. J. W. Barber


In [5]:
from scipy.sparse import coo_matrix

In [6]:
data['user']=data['user'].astype("category")
data['isbn']=data['isbn'].astype("category")

In [9]:
R= coo_matrix((data['ratings'].astype("float"), (data['user'].cat.codes.copy(), data['isbn'].cat.codes.copy())))

In [10]:
R.shape


(105283, 340556)

In [12]:
len(R.data)

1149780

In [17]:
M, N= R.shape
K=3

In [28]:
import numpy as np
P= np.random.rand(M, K)
Q= np.random.rand(K, N)

In [29]:
type(P)

numpy.ndarray

In [30]:
P.shape

(105283, 3)

In [31]:
Q.shape

(3, 340556)

In [34]:
from numpy.linalg import norm

def error(R, P, Q, lamda=0.02):
    Ratings= R.data
    rows= R.row
    cols= R.col
    e=0
    for ui in range(len(Ratings)):
        rui = Ratings[ui]
        u = rows[ui]
        i = cols[ui]
        if rui>0:
            e = e + pow((rui - np.dot(P[u,:], Q[:,i])), 2) + lamda*(  pow(norm(P[u,:]),2) + pow(norm(Q[:,i]),2))
    return e



In [36]:
error(R, P, Q)

21907574.608878028

In [37]:
rmse = np.sqrt(error(R, P, Q)/len(R.data))

In [38]:
rmse

4.365055547737966

In [53]:
def SGD(R, K, lamda=0.02, steps=10, gamma=0.001):
    M,N = R.shape
    P = np.random.rand(M,K)
    Q = np.random.rand(K,N)

    
    rmse = np.sqrt(error(R,P,Q,lamda)/len(R.data))
    print("Initial RMSE: "+str(rmse))
    
    for step in range(steps):
        for ui in range(len(R.data)):
            rui=R.data[ui]
            u = R.row[ui]
            i = R.col[ui]
            if rui>0:
                eui=rui-np.dot(P[u,:],Q[:,i])
                P[u,:]=P[u,:]+gamma*2*(eui*Q[:,i]-lamda*P[u,:])
                Q[:,i]=Q[:,i]+gamma*2*(eui*P[u,:]-lamda*Q[:,i])
        rmse = np.sqrt(error(R,P,Q,lamda)/len(R.data))
        if rmse<0.5:
            break
    print("Final RMSE: "+str(rmse))
    return P,Q

In [ ]:
P, Q = SGD(R, K=2, lamda=0.01, steps=30, gamma=0.0007)

Initial RMSE: 4.510664160024993
